Create a file with data interpolated from nearest grid points
===

Import required modules and set file names
----

In [1]:
from netCDF4 import Dataset,date2num
import pandas as pd
import numpy as np
from model import Model
from GCNet import GCNet

Filenames
---

In [2]:
suff = "19960101-20171231"
modelDataFN = "daily/ei.oper.an.sfc.regn128sc.tas."+suff+"_sub.nc"
modelMetaFN = "erai_geog_sub.nc"
newNCFN = "erai_tas_interp_"+suff+".nc"

Load/define ERAI metadata
----

In [3]:
M = Model(modelMetaFN)
M.loadMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

Load AWS metadata
----

In [4]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

Find closest grid points
-----

In [5]:
C = []
for awsIX in (range(nSites)):
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    C.append( closest )

Process tas data
----

In [6]:
print modelDataFN
D = Model(None, modelDataFN)

time, timeCF = D.loadData( "time" )
print time[0], time[-1]

tas = np.array( D.loadData( "tas" ) )
nRec, nLat, nLon = tas.shape
print nRec, nLat, nLon

Tpts = np.empty( (nRec, 4), dtype = 'float' )
TptsSave = np.empty( (nSites, nRec, 4), dtype = 'float' )
Tnew = np.empty( (nRec, nSites), dtype = 'float' )

for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
# for awsIX in ([0]):
    """ Basic AWS info """
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))

    """ Perform the interpolation on tas from model metadata """
    closest = C[awsIX]
    dx = "%.1f km" % closest[0][-1]
    print awsName, dx
    for r in range(nRec):
        Tpts[r, :] = [ tas[ r, d[0], d[1] ] for d in zip( closest['ixLat'], closest['ixLon'] )]
    Tnew[:,awsIX] = M.interpolate( Tpts, closest, awsLatLon )
    TptsSave[awsIX,:,:] = Tpts

daily/ei.oper.an.sfc.regn128sc.tas.19960101-20171231_sub.nc
1996-01-01 09:00:00 2017-12-31 09:00:00
8030 40 78
Swiss Camp 28.1 km
Crawford Point1 8.3 km
NASA-U 24.0 km
GITS 33.6 km
Humboldt 31.4 km
Summit 8.0 km
Tunu-N 26.0 km
DYE-2 19.2 km
JAR1 37.4 km
Saddle 36.2 km
South Dome 39.2 km
NASA-E 30.1 km
Crawford Point2 13.9 km
NGRIP 37.9 km
NASA-SE 23.0 km
KAR 14.0 km
JAR2 33.5 km
KULU 19.1 km
JAR3 32.8 km
Aurora 15.3 km
Petermann GL 37.2 km
Petermann ELA 30.2 km
NEEM 7.5 km


Create a netCDF file of the interpolated data
====

In [7]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset(newNCFN,mode='w',format='NETCDF4_CLASSIC') 
ncfile.title = "Interpolated temperature data"
ncfile.period = suff

# define dimensions
time_dim = ncfile.createDimension('time', None)
stn_dim = ncfile.createDimension('station', nSites)
pts_dim = ncfile.createDimension('pts', 4)

# define and write time
timeVar = ncfile.createVariable('time', np.float32, ('time',))
timeVar.units = timeCF.units
timeVar.calendar = timeCF.calendar
timeVar.standard_name = timeCF.standard_name
timeVar.long_name = timeCF.long_name
timeVar.axis = timeCF.axis

timeVar[:] = date2num(time, timeCF.units, timeCF.calendar)

# define and write tas
# tasVar = ncfile.createVariable('tas', np.float32, ('time','station'))
# tasVar = ncfile.createVariable('tas', np.float32, ('ensemble','time','station'))
tasVar = ncfile.createVariable('tas', np.float32, ('time','station'))
tasVar.units = "K"
tasVar.description = "Surface temperature, interpolated"
tasVar.long_name = "Reference height temperature"
# tasVar[:,:] = Tnew
tasVar[:] = Tnew

# define and write saved Tpts
ptsVar = ncfile.createVariable('Tpts', np.float32, ('station','time','pts'))
ptsVar.units = "K"
ptsVar.description = "Surface temperature, nearest grid points"
ptsVar.long_name = "Reference height temperature"
ptsVar[:] = TptsSave

ncfile.close()